In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import time
import os

from bs4 import BeautifulSoup
import re
import pandas as pd

import requests
import os
import urllib
from urllib.parse import urlparse

import subprocess


In [2]:
# prompt_message = "Press Enter to naviagte to: "
prompt_message = None
DELAY_SECONDS_CONTENT_LOADING = 2

In [3]:
IMPLICIT_DELAY = 60
EXPLICIT_DELAY = 30

In [4]:
#setup option for chrome profile
chrome_options = Options()
# chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")

#start web driver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.implicitly_wait(IMPLICIT_DELAY)

wait = WebDriverWait(driver, EXPLICIT_DELAY)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [5]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_element_by_xpath(xpath)

def sync_get_elements_by_xpath(xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_elements_by_xpath(xpath)


In [6]:
def sign_in():
    email = sync_get_element_by_xpath("//input[@data-testid='signin-email']")
    password = sync_get_element_by_xpath("//input[@data-testid='signin-password']")

    email.send_keys("huohsien@gmail.com")
    password.send_keys("jj1216")

#     wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb")))
#     sign_in_btn = driver.find_element_by_class_name("button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb") 
    sign_in_btn = sync_get_element_by_xpath("//span[contains(text(), 'Sign In')]/..")
    sign_in_btn.click()
    
def check_lesson_complete(item)->bool:
    e1 = item.find_element_by_xpath("./div/div")
    c = e1.get_attribute('class')
    if c == 'index--lesson-card--mwX1V index--card--3DZMr shared--card--3X88h':
        return True
    else:
        return False

def expand_all_complete_lessons():
    #click open the ones that were marked Complete
    lessons = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")
    for lesson in lessons:
        if check_lesson_complete(lesson):
            lesson.click()
    element = driver.find_element_by_xpath("//*[h3='lesson 1']")
    driver.execute_script("return arguments[0].scrollIntoView(true);", element)
#     time.sleep(1)

def embed_tag(tag, html_str):
    return '<{}>'.format(tag) + html_str + '</{}>'.format(tag)

def debug_breakpoint(title, message=None):
    if message == None:
        print(title)
    else:
        input(title + ' - ' + message)

def write_to_clipboard(output):
    process = subprocess.Popen(
        'pbcopy', env={'LANG': 'en_US.UTF-8'}, stdin=subprocess.PIPE)
    process.communicate(output.encode('utf-8'))

In [7]:
def download_css_files():
    
    css_folder_path = os.path.join(root_folder, topic_name,lesson_name, part_name, "css")
    
    if not os.path.exists(css_folder_path):
        os.makedirs(css_folder_path)
        
    html_src = driver.page_source
    base = driver.current_url
    soup = BeautifulSoup(html_src)
    css_link_elms = soup.find_all('link', {'rel': 'stylesheet'})
    
    css_links = []
    for idx, css_link_elm in enumerate(css_link_elms, start = 1):
        link = css_link_elm['href']
        if link.startswith('/'):
            link = urllib.parse.urljoin(base, link)
        css_links.append(link)
#         print(idx, ": ", link)
    
    for css_link in css_links:
        r = requests.get(css_link, allow_redirects=True)
        a = urlparse(css_link)
        file_name = os.path.basename(a.path)
        file_name = os.path.join(css_folder_path, file_name)
#         print("file_name= ", file_name) 
    
        open(file_name, 'wb').write(r.content)

def make_images_local(soup):
    
    images_folder = os.path.join(root_folder, topic_name,lesson_name, part_name, "img")

    img_elms = soup.find_all('img')
    for img_elm in img_elms:
        img_link = img_elm['src']
        r = requests.get(img_link, allow_redirects=True)
        a = urlparse(img_link)
        file_name = os.path.basename(a.path)                
        img_path = "./" + images_folder + "/" + file_name

        if not os.path.exists(images_folder):
            os.makedirs(images_folder)

        open(img_path, 'wb').write(r.content)
#         print("file_name: ", file_name)
        # replace src in img element
        img_elm['src'] = "./img/{}".format(file_name)

def create_head_str():
    
    css_folder_path = os.path.join(root_folder, topic_name,lesson_name, part_name, "css")
    css_filenames = [f for f in os.listdir(css_folder_path) if os.path.isfile(os.path.join(css_folder_path, f))]
    
    page_html = driver.page_source
    page_soup = BeautifulSoup(page_html, 'html.parser')
    style_elms = page_soup.find('head').find_all('style')
    
    meta_elm = '<meta http-equiv="Content-type" content="text/html; charset=utf-8">'
    head_str = '<head>' + meta_elm
    for css_filename in css_filenames:
        link_str = '<link rel="stylesheet" href="./css/{}">'.format(css_filename)
        head_str = head_str + link_str
    for style_elm in style_elms:
        head_str = head_str + str(style_elm)
    
    head_str = head_str + '</head>'
#     print("head_str: ", head_str)
    return head_str

def process_content_html(content_html_str):
    
    download_css_files()
            
    head_str = create_head_str()
    body_str = embed_tag('body', content_html_str)
    page_html_str = head_str + body_str
    html_str = embed_tag('html', page_html_str)
    
    # remove unwanted div element
    soup = BeautifulSoup(html_str, 'html.parser')
    div_to_be_remove_class_name = '_main--notification-bar--2AVbT'
    to_be_removed_elms = soup.find_all('div',class_=div_to_be_remove_class_name)
    for to_be_removed_elm in to_be_removed_elms:
        to_be_removed_elm.decompose()
    
    make_images_local(soup)
    
    return soup.prettify()

def copy_youtube_link_to_clipboard(video_link):
    write_to_clipboard(video_link)
    time.sleep(1)
    
def grab_all_youtube_links(content_html_str, download=False):

    content_soup = BeautifulSoup(content_html_str, 'html.parser')
    youtube_iframe_elms = content_soup.find_all('iframe', {'title': 'YouTube video player'})
    video_links = ''
    for idx, youtube_iframe_elm in enumerate(youtube_iframe_elms, start=1):
        video_link = youtube_iframe_elm['src']
        video_link = video_link + '#index={}_{}_{}_{}'.format(topic_name,lesson_name, part_name,idx)
        video_links = video_links + video_link + '\n\n'
        if download == True:
            copy_youtube_link_to_clipboard
        
    #     print(idx, ' : ', video_link)
#     print(video_links)
    with open('all_youtube_video_links.txt','a') as fp:
        fp.writelines(video_links)

In [8]:
def save_processed_html():

    contents = sync_get_elements_by_xpath("//div[@id='content']")
    assert len(contents) == 1

    content_html = contents[0].get_attribute('outerHTML')
    html_str = process_content_html(content_html)
    grab_all_youtube_links(content_html, False)
    
    file_name = os.path.join(root_folder, topic_name,lesson_name, part_name, "content.html")
    with open(file_name, 'w') as f:
        f.write(html_str)
#     print("html_str= ", html_str)

In [9]:
root_address = 'https://classroom.udacity.com/nanodegrees/nd101/dashboard/overview'
root_folder = './dlnd_course_downloaded'

In [10]:
# start
driver.get(root_address)
os.makedirs(root_folder, exist_ok=True)

In [11]:
if check_exists_by_xpath("//h1[contains(text(), 'Sign in to your account')]"):
    print("Signing in...")
    sign_in()

Signing in...


In [12]:
# manually close pop up ads and agree the agreement so on so forth
input("In the curriculum page?[y/n]")

In the curriculum page?[y/n]y


'y'

In [13]:
# Prepare loop through all topics
topic_elms = sync_get_elements_by_xpath("//*[@id='tree-core-curriculum']/*")
num_topics = len(topic_elms)

# iterate each topic
for idx_topics in range(num_topics):
    
    # Create topic folder
    topic_name = ''.join(topic_elms[idx_topics].text.split('\n')[0:2])                   
    os.makedirs(os.path.join(root_folder, topic_name), exist_ok=True)
    
    debug_breakpoint("Topic {}".format(topic_name), prompt_message)
    
    # click into a topic
    link_topic = topic_elms[idx_topics].find_element_by_xpath("./a")
    link_topic.click()
    
    # Prepare loop through all lessons
    expand_all_complete_lessons()
    
    lesson_elms = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")
    num_lessons = len(lesson_elms)
    
    #iterate each lesson
    for idx_lesson in range(num_lessons):

        # Create lesson folder
        lesson_elm = lesson_elms[idx_lesson]
        lesson_name = ' - '.join(lesson_elm.text.split('\n')[1:3])                   
        os.makedirs(os.path.join(root_folder, topic_name,lesson_name), exist_ok=True)

        debug_breakpoint("Lesson {}".format(lesson_name), prompt_message)

        # click into a lesson
        link_lesson = lesson_elm.find_element_by_xpath("./div/div/a")
        link_lesson.click()
        
        # Prepare loop through all parts of a lesson
        part_elms = sync_get_elements_by_xpath("//*[@id='tree-concepts']/ol/*")
        num_parts = len(part_elms)
        
        # iterate each part
        for idx_part in range(num_parts):
            
            # Create part folder
            part_elm = part_elms[idx_part]
            part_name = part_elm.text
            os.makedirs(os.path.join(root_folder, topic_name,lesson_name, part_name), exist_ok=True)
            
            debug_breakpoint("Lesson part {}".format(part_name), prompt_message)
            
            # click into a part
            link_part = part_elm.find_element_by_xpath("./a")
            link_part.click()
            
            # REAL CONTENTS HERE to Be STORED!
            #
            time.sleep(DELAY_SECONDS_CONTENT_LOADING)
            
            save_processed_html()
#             debug_breakpoint(driver.current_url, prompt_message)
            
            # reflesh webElement after webpage reload
            part_elms = sync_get_elements_by_xpath("//*[@id='tree-concepts']/ol/*")

        # go back to Lesson List
        back_link = sync_get_element_by_xpath("//h3[@class='_sidebar--header--2BaH_']/a") 
        back_link.click()
        
        # click open the completed lessons
        expand_all_complete_lessons()
        
        # reflesh webElement after webpage reload
        lesson_elms = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")

    # reflesh webElement after webpage reload
    topic_elms = sync_get_elements_by_xpath("//*[@id='tree-core-curriculum']/*")
    

Topic 1. Introduction to Deep Learning
Lesson LESSON 1 - Welcome to the Deep Learning Nanodegree Program
Lesson part 1. Welcome to the Deep Learning Nanodegree Program
Lesson part 2. Meet Your Instructors
Lesson part 3. Program Structure
Lesson part 4. Community Guidelines
Lesson part 5. Prerequisites
Lesson part 6. Getting Set Up
Lesson LESSON 2 - Nanodegree Career Services
Lesson part 1. Access the Career Portal
Lesson LESSON 3 - Knowledge, Community, and Careers
Lesson part 1. Getting Help
Lesson part 2. Knowledge
Lesson part 3. Student Hub
Lesson part 4. Community Initiatives
Lesson part 5. Access the Career Portal
Lesson LESSON 4 - Get Help with Your Account
Lesson part 1. FAQ
Lesson part 2. Support
Lesson LESSON 5 - Anaconda
Lesson part 1. Instructor
Lesson part 2. Introduction
Lesson part 3. What is Anaconda?
Lesson part 4. Installing Anaconda
Lesson part 5. Managing packages
Lesson part 6. Managing environments
Lesson part 7. More environment actions
Lesson part 8. Best practic

Lesson part 31. Capsule Networks
Lesson part 32. Increasing Depth
Lesson part 33. CNNs for Image Classification
Lesson part 34. Convolutional Layers in PyTorch
Lesson part 35. Feature Vector
Lesson part 36. Pre-Notebook: CNN Classification
Lesson part 37. Notebook: CNNs for CIFAR Image Classification
Lesson part 38. CIFAR Classification Example
Lesson part 39. CNNs in PyTorch
Lesson part 40. Image Augmentation
Lesson part 41. Augmentation Using Transformations
Lesson part 42. Groundbreaking CNN Architectures
Lesson part 43. Visualizing CNNs (Part 1)
Lesson part 44. Visualizing CNNs (Part 2)
Lesson part 45. Summary of CNNs
Lesson part 46. Introduction to GPU Workspaces
Lesson part 47. Workspace Playground
Lesson part 48. GPU Workspace Playground
Lesson LESSON 2 - GPU Workspaces Demo
Lesson part 1. Introduction to GPU Workspaces
Lesson part 2. Workspace Playground
Lesson part 3. GPU Workspace Playground
Lesson LESSON 3 - Cloud Computing
Lesson part 1. AWS Overview
Lesson part 2. AWS Setu

Lesson part 10. Word2Vec Model
Lesson part 11. Model & Validations
Lesson part 12. Negative Sampling
Lesson part 13. Pre-Notebook: Negative Sampling
Lesson part 14. Notebook: Negative Sampling
Lesson part 15. SkipGramNeg, Model Definition
Lesson part 16. Complete Model & Custom Loss
Lesson LESSON 6 - Sentiment Prediction RNN
Lesson part 1. Sentiment RNN, Introduction
Lesson part 2. Pre-Notebook: Sentiment RNN
Lesson part 3. Notebook: Sentiment RNN
Lesson part 4. Data Pre-Processing
Lesson part 5. Encoding Words, Solution
Lesson part 6. Getting Rid of Zero-Length
Lesson part 7. Cleaning & Padding Data
Lesson part 8. Padded Features, Solution
Lesson part 9. TensorDataset & Batching Data
Lesson part 10. Defining the Model
Lesson part 11. Complete Sentiment RNN
Lesson part 12. Training the Model
Lesson part 13. Testing
Lesson part 14. Inference, Solution
Lesson PROJECT - Project: Generate TV Scripts
Lesson part 1. Introduction
Lesson part 2. GPU Workspaces: Best Practices
Lesson part 3. Pr

ElementClickInterceptedException: Message: element click intercepted: Element <a class="index--curtain--3sKxm shared--curtain--2_FSP" href="/nanodegrees/nd101/parts/2ea78ff8-befd-4046-b06e-5327871b0748/modules/72f47c70-f4f0-49f8-a67a-a3562f0bd7ac/lessons/de9a07cd-bfb4-4d09-a305-2f20f158b965/concepts/last-viewed">...</a> is not clickable at point (760, 132). Other element would receive the click: <div class="_header-content--header--3mdiS shared--outer-container--3eppq">...</div>
  (Session info: chrome=86.0.4240.80)


In [ ]:
# link_topic.click() 

In [ ]:
# expand_all_complete_lessons()



In [ ]:
# element = driver.find_element_by_xpath("//*[h3='lesson 1']")
# driver.execute_script("return arguments[0].scrollIntoView(true);", element)